# Task 3
This serves as a template which will guide you through the implementation of this task. It is advised to first read the whole template and get a sense of the overall structure of the code before trying to fill in any of the TODO gaps.
This is the jupyter notebook version of the template. For the python file version, please refer to the file `template_solution.py`.

First, we import necessary libraries:

In [19]:
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import os
import torch
from torchvision import transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision.models import resnet152, ResNet152_Weights
from sklearn.model_selection import KFold
from torch.optim import Adam
# Add any other imports you need here
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
# The device is automatically set to GPU if available, otherwise CPU
# If you want to force the device to CPU, you can change the line to
# device = torch.device("cpu")
# When using the GPU, it is important that your model and all data are on the
# same device.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [21]:
# Global Variables
global_batch_size = 50
global_num_workers = 8
#global_model = models.resnet18(weights='IMAGENET1K_V1')
global_model = resnet152(weights=ResNet152_Weights.IMAGENET1K_V2)
global_embedding_size = 512

Downloading: "https://download.pytorch.org/models/resnet152-f82ba261.pth" to /root/.cache/torch/hub/checkpoints/resnet152-f82ba261.pth
100%|██████████| 230M/230M [00:04<00:00, 59.5MB/s]


In [22]:
"""
Transform, resize and normalize the images and then use a pretrained model to extract
the embeddings.
"""
# TODO: define a transform to pre-process the images
# The required pre-processing depends on the pre-trained model you choose
# below.
# See https://pytorch.org/vision/stable/models.html#using-the-pre-trained-models
"""train_transforms = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Resize((224, 224)),  # Resize images to (224, 224)
    #transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
    #transforms.Normalize([0.6110, 0.5012, 0.3752], [0.2575, 0.2659, 0.2801])

])"""
train_transforms = ResNet152_Weights.IMAGENET1K_V2.transforms

train_dataset = datasets.ImageFolder(root="/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/dataset/", transform=train_transforms)
# Hint: adjust batch_size and num_workers to your PC configuration, so that you don't
# run out of memory (VRAM if on GPU, RAM if on CPU)
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=64,
                          shuffle=False,
                          pin_memory=True,
                          num_workers=8)

# TODO: define a model for extraction of the embeddings (Hint: load a pretrained model,
# more info here: https://pytorch.org/vision/stable/models.html)
model = global_model
model.eval()
#for param in model.parameters():
        #param.requires_grad = False
model.fc = nn.Sequential()
model.to(device)
"""
embedding_size = global_embedding_size
num_images = len(train_dataset)
embeddings = np.zeros((num_images, embedding_size))

# pick your model
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])
"""

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


'\nembedding_size = global_embedding_size\nnum_images = len(train_dataset)\nembeddings = np.zeros((num_images, embedding_size))\n\n# pick your model\nfeature_extractor = torch.nn.Sequential(*list(model.children())[:-1])\n'

In [ ]:
# This created embeddings.npy file...
"""
with torch.no_grad():
  start_idx = 0
  for inputs, _ in train_loader:
    inputs = inputs.to(device)
    outputs = feature_extractor(inputs)
    print(outputs.shape)
    embed = outputs.resize_(outputs.shape[0], outputs.shape[1])
    print(embed.shape)
    batch_size = outputs.shape[0]
    embeddings[start_idx:start_idx+batch_size] = outputs
    start_idx += batch_size
    print(start_idx)
print(embeddings.shape)
print(embeddings)
np.save('/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/embeddings.npy', embeddings)
"""

In [ ]:
# This created embeddings2.npy file
embeddings = []
for features, labels in train_loader:
        embeddings.append(model(features).detach().numpy())
np.save('/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/embeddings2.npy', embeddings)

In [30]:
def get_data(file, train=True):
    """
    Load the triplets from the file and generate the features and labels.

    input: file: string, the path to the file containing the triplets
          train: boolean, whether the data is for training or testing

    output: X: numpy array, the features
            y: numpy array, the labels
    """
    triplets = []
    with open(file) as f:
        for line in f:
            triplets.append(line)

    # generate training data from triplets
    train_dataset = datasets.ImageFolder(root="/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/dataset/",
                                         transform=None)
    filenames = [s[0].split('/')[-1].replace('.jpg', '') for s in train_dataset.samples]
    embeddings = np.load('/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/embeddings.npy')
    #embeddings = np.load('/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/embeddings2.npy')
    # TODO: Normalize the embeddings
    embeddings /= np.linalg.norm(embeddings, axis=1, keepdims=True)
    #embeddings = np.swapaxes(embeddings, 1, 2)
    #embeddings = (embeddings - np.mean(embeddings, axis=0)) / np.std(embeddings, axis=0)
    file_to_embedding = {}
    for i in range(len(filenames)):
        file_to_embedding[filenames[i]] = embeddings[i]
    """for i in range(200):
        for j in range(50):
            file_to_embedding[filenames[j+i*50]] = embeddings[i][j]"""
    X = []
    y = []
    # use the individual embeddings to generate the features and labels for triplets
    for t in triplets:
        emb = [file_to_embedding[a] for a in t.split()]
        X.append(np.hstack([emb[0], emb[1], emb[2]]))
        y.append(1)
        # Generating negative samples (data augmentation)
        if train:
            X.append(np.hstack([emb[0], emb[2], emb[1]]))
            y.append(0)
    X = np.vstack(X)
    y = np.hstack(y)
    return X, y

Hint: adjust batch_size and num_workers to your PC configuration, so that you don't run out of memory (VRAM if on GPU, RAM if on CPU)

In [31]:
def create_loader_from_np(X, y=None, train=True, batch_size=64, shuffle=True, num_workers=8):
    if train:
        dataset = TensorDataset(torch.from_numpy(X).type(torch.float),
                                torch.from_numpy(y).type(torch.long))
    else:
        if y is not None:
            dataset = TensorDataset(torch.from_numpy(X).type(torch.float),
                                    torch.from_numpy(y).type(torch.long))
        else:
          dataset = TensorDataset(torch.from_numpy(X).type(torch.float))
    loader = DataLoader(dataset=dataset,
                        batch_size=batch_size,
                        shuffle=shuffle,
                        pin_memory=True, num_workers=num_workers)
    return loader


TODO: define a model. Here, the basic structure is defined, but you need to fill in the details

In [32]:
class Net(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 64)
        self.fc4 = nn.Linear(64, 1)
        #self.fc1 = nn.Linear(input_size, 512)
        #self.fc2 = nn.Linear(512, 256)
        #self.fc4 = nn.Linear(256, 1, bias=True)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        #x = F.relu(self.fc4(x))
        return x

In [26]:
"""
TRAIN_TRIPLETS = '/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/train_triplets.txt'

# load the training data
X, y = get_data(TRAIN_TRIPLETS)
# Create data loaders for the training data
train_loader = create_loader_from_np(X, y, train = True, batch_size=64)
# delete the loaded training data to save memory, as the data loader copies
#del X
#del y
"""

In [27]:
"""
TEST_TRIPLETS = '/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/test_triplets.txt'

# repeat for testing data
X_test, y_test = get_data(TEST_TRIPLETS, train=False)
test_loader = create_loader_from_np(X_test, train = False, batch_size=2048, shuffle=False)
#del X_test
#del y_test
"""

In [34]:
def test_model(model, loader):
    model.eval()
    predictions = []
    # Iterate over the test data
    with torch.no_grad(): # We don't need to compute gradients for testing
        for [x_batch] in loader:
            x_batch= x_batch.to(device)
            predicted = model(x_batch)
            predicted = predicted.cpu().numpy()
            # Rounding the predictions to 0 or 1
            predicted[predicted >= 0.5] = 1
            predicted[predicted < 0.5] = 0
            predictions.append(predicted)
        predictions = np.vstack(predictions)
    np.savetxt("/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/results_task3_6.txt", predictions, fmt='%i')

In [ ]:
"""
#ursprüngliche Version
def test_model(model, loader):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = F.binary_cross_entropy(outputs.squeeze(), labels.float(), reduction='sum')
            total_loss += loss.item()
            predicted = (outputs >= 0.5).int()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    loss = total_loss / total
    acc = correct / total
    return loss, acc
"""

In [ ]:
"""
#ursprüngliche Version
def train_model(train_loader, val_loader):
    model = Net(3 * 512)
    model.train()
    model.to(device)

    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    n_epochs = 5

    for epoch in range(n_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{n_epochs}, Train Loss: {epoch_loss:.4f}")

        # Validate the model
        val_loss, val_acc = test_model(model, val_loader)
        print(f"Epoch {epoch+1}/{n_epochs}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    return model
"""

In [41]:
def train_model(train_loader):
    """
    The training procedure of the model; it accepts the training data, defines the model
    and then trains it.

    input: train_loader: torch.data.util.DataLoader, the object containing the training data

    output: model: torch.nn.Module, the trained model
    """
    model = Net(3*512)
    model.train()
    model.to(device)
    n_epochs = 5
    # TODO: define a loss function, optimizer and proceed with training. Hint: use the part
    # of the training data as a validation split. After each epoch, compute the loss on the
    # validation split and print it out. This enables you to see how your model is performing
    # on the validation data before submitting the results on the server. After choosing the
    # best model, train it on the whole training data.
    for epoch in range(n_epochs):
        for [X, y] in train_loader:
            pass
    return model

In [ ]:
"""
kf = KFold(n_splits=2, shuffle=True, random_state=42)

val_losses = []
val_accuracies = []

for fold, (train_index, val_index) in enumerate(kf.split(X)):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    train_loader = create_loader_from_np(X_train, y_train, train=True, batch_size=64)
    val_loader = create_loader_from_np(X_val, y_val, train=False, batch_size=64, shuffle=False)

    model = train_model(train_loader, val_loader)

    val_loss, val_acc = test_model(model, val_loader)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    print(f"Fold {fold+1}: Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

print(f"Average Validation Loss: {np.mean(val_losses):.4f}, Average Validation Accuracy: {np.mean(val_accuracies):.4f}")
"""


In [42]:
# load the training data
TRAIN_TRIPLETS = '/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/train_triplets.txt'
TEST_TRIPLETS = '/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/test_triplets.txt'

X, y = get_data(TRAIN_TRIPLETS)
# Create data loaders for the training data
train_loader = create_loader_from_np(X, y, train = True, batch_size=64)
# delete the loaded training data to save memory, as the data loader copies
del X
del y

# repeat for testing data
X_test, y_test = get_data(TEST_TRIPLETS, train=False)
test_loader = create_loader_from_np(X_test, train = False, batch_size=2048, shuffle=False)
del X_test
del y_test

# define a model and train it
model = train_model(train_loader)

# test the model on the test data
test_model(model, test_loader)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
